In [1]:
import numpy as np
import pandas as pd

In [2]:
pl_df = pd.read_csv('this_one_problem_logs_seq.csv')

/usr/local/lib/python2.7/site-packages/IPython/core/interactiveshell.py:2717: DtypeWarning: Columns (21,23,25) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [37]:
# read 9 experiments
problem_set = 303899
ps_df = pd.read_csv('../22-Experiments/'+str(problem_set)+'_exp.csv', header=None)

In [38]:
print len(ps_df[1].unique())
print len(ps_df[ps_df[2]==1])
print len(ps_df[ps_df[2]==0])

928
469
459


In [39]:
train_df = pl_df[pl_df['user_id'].isin(ps_df[1].unique().tolist())].reset_index()

In [40]:
print(len(train_df['user_id'].unique()))

928


In [41]:
# read ps_list
ps_list = []
with open('ps_list') as file:
    for line in file:
        line = line.strip()
        ps_list.append(int(line))

In [42]:
encode_list = []
for index, row in train_df.iterrows():
    encode = np.zeros(len(ps_list)+1)
    try:
        idx = ps_list.index(row['sequence_id'])
        encode[idx] = 1
    except ValueError:
        encode[len(ps_list)] = 1

    encode_list.append(encode)
encode_array = np.stack(encode_list, axis=0)

In [43]:
final_df = pd.DataFrame(encode_array, columns=ps_list+['-'])
# u'correct , u'bottom_hint, u'hint_count, u'attempt_count, u'first_response_time
final_df['correct'] = train_df['correct']
final_df['bottom_hint'] = train_df['bottom_hint']
final_df['hint_count'] = train_df['hint_count']
final_df['attempt_count'] = train_df['attempt_count']
final_df['first_response_time'] = train_df['first_response_time']

norm = lambda x: (x - x.min()) / (x.max() - x.min())
final_df.insert(len(final_df.columns), 'NofFRT', train_df.groupby('sequence_id')['first_response_time'].transform(norm))
del final_df['first_response_time']

final_df['user_id'] = train_df['user_id']
final_df['id'] = train_df['id']
final_df['NofFRT'].fillna(0.5, inplace=True)
final_df.isnull().values.any()

False

In [44]:
final_df.to_csv(str(problem_set)+'_train_data.csv', index=False)